In [41]:
import pandas as pd 



In [42]:
df = pd.read_csv("cleaned_ice_store_data.csv")

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2614 entries, 0 to 2613
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Day           2562 non-null   object
 1   Date          2562 non-null   object
 2   Grand Sales   2562 non-null   object
 3   Special Note  657 non-null    object
 4   High Temp     2562 non-null   object
 5   Low Temp      2562 non-null   object
 6   Flavor        2562 non-null   object
 7   Made          2561 non-null   object
 8   Used          2561 non-null   object
dtypes: object(9)
memory usage: 183.9+ KB


In [43]:
num_cols = ["High Temp", "Low Temp", "Made", "Used"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
df["Date"] = pd.to_datetime(df["Date"])
df["High Temp"] = pd.to_numeric(df["High Temp"], errors='coerce')
df["Low Temp"] = pd.to_numeric(df["Low Temp"], errors='coerce')
df["Made"] = pd.to_numeric(df["Made"], errors='coerce')
df["Used"] = pd.to_numeric(df["Used"], errors='coerce')
df["Flavor"] = df["Flavor"].astype(str)


/var/folders/wj/mwd6zf295nj8ld04tk35rd240000gn/T/ipykernel_67001/260629718.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


In [44]:
df["Usage_Efficiency"] = df["Used"]/df["Made"]


In [45]:
df["Waste"] = df["Made"] - df["Used"]
df["Waste_Percentage"] = df["Waste"] / df["Made"]

In [46]:
df["Month"] = df["Date"].dt.month
df["Week"] = df["Date"].dt.isocalendar().week
df["Year"] = df["Date"].dt.year
df["Weekday"] = df["Date"].dt.day_name()




In [47]:
sales_by_day = df.groupby("Date")["Grand Sales"].sum().reset_index()
flavor_demand = df.groupby("Flavor")["Used"].sum().reset_index()
df["Grand Sales"] = (df["Grand Sales"].replace(",","",regex=True))
df["Grand Sales"] = pd.to_numeric(df["Grand Sales"], errors='coerce')
weather_correlation = df[["High Temp", "Low Temp", "Grand Sales"]].corr()

# One-hot encode special notes
event_dummies = pd.get_dummies(df["Special Note"], prefix="Event")
df_event = pd.concat([df, event_dummies], axis=1)

# Correlation only on numeric columns
corr_numeric = df_event.select_dtypes(include=["number"]).corr()

specialNote_correlation = corr_numeric["Grand Sales"].sort_values(ascending=False)

specialNote_correlation

Grand Sales         1.000000
Low Temp            0.508592
High Temp           0.491704
Used                0.267293
Made                0.241526
Usage_Efficiency    0.108996
Week                0.044088
Month               0.033419
Waste               0.006074
Waste_Percentage   -0.108996
Year                     NaN
Name: Grand Sales, dtype: float64

In [48]:
df.to_csv("transformed_ice_store_data.csv", index=False)

In [50]:
df.head()
df.describe()
df.isnull().sum()

Day                   52
Date                  52
Grand Sales           73
Special Note        1957
High Temp             73
Low Temp              73
Flavor                 0
Made                  74
Used                  74
Usage_Efficiency      75
Waste                 75
Waste_Percentage      75
Month                 52
Week                  52
Year                  52
Weekday               52
dtype: int64